In [1]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict
import cv2
import json
from cloudvolume import CloudVolume
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
import shutil

In [2]:
# define some paths and variables
animal = 'DK46'
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)
# local imports
from lib.sqlcontroller import SqlController
from lib.file_location import FileLocationManager
from lib.utilities_cvat_neuroglancer import NumpyToNeuroglancer, calculate_chunks
fileLocationManager = FileLocationManager(animal)
sqlController = SqlController(animal)
from lib.utilities_process import SCALING_FACTOR
# vars
sections = sqlController.get_sections(animal, 1)
num_sections = len(sections)
width = sqlController.scan_run.width
height = sqlController.scan_run.height
scale_xy = sqlController.scan_run.resolution
z_scale = sqlController.scan_run.zresolution
scales = np.array([scale_xy, scale_xy, z_scale])
width = int(width * SCALING_FACTOR)
height = int(height * SCALING_FACTOR)
aligned_shape = np.array((width, height))

In [3]:
#### fetch the data
rows = sqlController.get_annotations(animal, 1, 'cerebellum')
polygons = defaultdict(list)

for row in rows:
    xy = (row.x/scale_xy, row.y/scale_xy)
    z = int(np.round(row.z/z_scale))
    polygons[z].append(xy)

In [4]:
#### loop through all the sections and write to a template, then add that template to the volume
volume = np.zeros((aligned_shape[1], aligned_shape[0], num_sections), dtype=np.uint8)
color = 100
for section,points in polygons.items():
    template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
    points = np.array(points)
    points = np.round(points*SCALING_FACTOR)
    points = points.astype(np.int32)
    cv2.polylines(template, [points], True, color, 8, lineType=cv2.LINE_AA)
    volume[:, :, section - 1] = template

In [5]:
#### set variables for mesh/segmentation. the scales are for a regular neurotrace 0.325nm,0.325nm,20um brain
scales = (10400, 10400, 20000)
# Voxel offset
offset = (0, 0, 0)
data_type = str(volume.dtype)
layer_type = 'segmentation'
chunks = [64, 64, 64]
num_channels = 1
downsample = True

OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'mesh')

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [6]:
#### initialize the Cloudvolume
cloudpath = f'file://{OUTPUT_DIR}'
info = CloudVolume.create_new_info(
    num_channels = num_channels,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = scales, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = chunks, # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 4560/4560 [00:35<00:00, 127.18it/s]


In [7]:
#### create json for neuroglancer info files
cv2 = CloudVolume(cloudpath, 0)
cv2.info['segment_properties'] = 'names'
cv2.commit_info()

segment_properties_path = os.path.join(cloudpath.replace('file://', ''), 'names')
os.makedirs(segment_properties_path, exist_ok=True)

info = {
    "@type": "neuroglancer_segment_properties",
    "inline": {
        "ids": [str(color)],
        "properties": [{
            "id": "label",
            "type": "label",
            "values": [str(color)]
        }]
    }
}
with open(os.path.join(segment_properties_path, 'info'), 'w') as file:
    json.dump(info, file, indent=2)


In [8]:
#### 1st create mesh
mse = 40
tq = LocalTaskQueue(parallel=1)
mesh_dir = f'mesh_mip_0_err_{mse}'
cv2.info['mesh'] = mesh_dir
cv2.commit_info()
tasks = tc.create_meshing_tasks(cv2.layer_cloudpath, mip=0, mesh_dir=mesh_dir, max_simplification_error=mse)
tq.insert(tasks)
tq.execute()


Downloading:  12%|█▏        | 61/512 [00:00<00:00, 479.21it/s]


Downloading: 100%|██████████| 512/512 [00:01<00:00, 10.38it/s]


Downloading:  64%|██████▎   | 326/512 [00:00<00:04, 44.19it/s]


Downloading:  46%|████▌     | 233/512 [00:00<00:00, 452.22it/s]


Downloading:  95%|█████████▍| 486/512 [00:01<00:02, 10.80it/s]


Tasks:  47%|████▋     | 14/30 [01:26<01:12,  4.53s/it]


Tasks:  90%|█████████ | 27/30 [01:30<00:00,  3.67it/s]


Tasks: 100%|██████████| 30/30 [01:30<00:00,  3.02s/it]


In [ ]:
##### 2nd mesh task, create manifest
tasks = tc.create_mesh_manifest_tasks(cv2.layer_cloudpath, mesh_dir=mesh_dir)
tq.insert(tasks)
tq.execute()


Tasks:  95%|█████████▍| 947/999 [00:32<00:01, 45.05it/s]